In [1]:
from gurobipy import Model, GRB
import time
import pandas as pd
import numpy as np
#from pylinerating.ieee738 import thermal_rating
#from pylinerating import conductor
import matplotlib.pyplot as plt
from tqdm import tqdm
plt.rcParams["figure.figsize"] = (20,8)
import pickle
import json
from IPython.display import Image
from tqdm import tqdm
import pickle

In [2]:
# unserved_energy_per_year = {}
# for load_increase in load_increases: load increase ex: 1.02, 1.04
#     unserved_energy_per_year[load_increase] = {}
#     for SOC_opt in BESS_capacities: BESS capacities in GWmin
#         unserved_energy_per_year[load_increase][SOC_opt] = {} year as a float : unserved energy in GWmin, ex: '9': 0.08,

# service_life = {}
# for load_increase in load_increases: load increase ex: 1.02, 1.04
#     service_life[load_increase] = {}
#     for SOC_opt in BESS_capacities: BESS capacities in GWmin
#         service_life[load_increase][SOC_opt] = {}
#         for soh in ['SOH_80', 'SOH_60', 'SOH_40']: State of Health, ex: 'SOH_80', 'SOH_60', 'SOH_40'
#             service_life[load_increase][SOC_opt][soh] = np.nan service life in years, ex: 1, 1.5

# unserved_energy = {}
# for load_increase in load_increases: load increase ex: 1.02, 1.04
#     unserved_energy[load_increase] = {}
#     for SOC_opt in BESS_capacities: BESS capacities in GWmin
#         unserved_energy[load_increase][SOC_opt] = {}
#         for soh in ['SOH_80', 'SOH_60', 'SOH_40']: State of Health, ex: 'SOH_80', 'SOH_60', 'SOH_40'
#             unserved_energy[load_increase][SOC_opt][soh] = np.nan unserved energy by the time soh is reached in GWmin

In [3]:
with open('service_life.pkl', 'rb') as outfile:
    service_life = pickle.load(outfile)

In [4]:
with open('unserved_energy_per_year.pkl', 'rb') as outfile:
    unserved_energy_per_year = pickle.load(outfile)

In [5]:
with open('unserved_energy.pkl', 'rb') as outfile:
    unserved_energy = pickle.load(outfile)

In [6]:
load_increase = 1.02

BESS_capacities = list(service_life[load_increase].keys())

for SOC_opt in BESS_capacities:
    x = list(unserved_energy_per_year[load_increase][SOC_opt].keys())
    y = (1000/60) * np.array(list(unserved_energy_per_year[load_increase][SOC_opt].values()))
    plt.plot(x, y, label = str(round((1000/60)*SOC_opt, 2)) + ' MWh BESS')
    plt.xlim([9,20])
    plt.xlabel('Year [-]', fontsize = 20)
    plt.ylabel('Unserved Energy [MWh]', fontsize = 20)
    plt.title('Unserved Energy Per Year [MWh] vs Year [-] For BESS Capacities [MWh]', fontsize = 20)
    plt.legend(fontsize = 10);

In [7]:
load_increases = list(unserved_energy.keys())

y_offset = 0.5
round_dec_pl = 1

for ii, soh in enumerate(['SOH_80', 'SOH_60', 'SOH_40']):
    for load_increase in load_increases:
        
        x = (1000/60) * np.array(list(service_life[load_increase].keys()))
        y = []
        for SOC_opt in BESS_capacities:
            y.append(service_life[load_increase][SOC_opt][soh])
        
        if soh == 'SOH_80':
            plt.plot(x, y, '-x', markersize=10, label = str(load_increase) + 'x | SOH~80')
            for ii, SOC_opt in enumerate(x):
                plt.annotate(str(round((1000/60) * unserved_energy[load_increase]\
                                       [np.array(list(service_life[load_increase].keys()))[ii]][soh],
                                       round_dec_pl))+' MWh', 
                             (x[ii] + 0.01, y[ii] - y_offset), 
                             fontsize = 15)
        if soh == 'SOH_60':
            plt.plot(x, y, '-o', markersize=10, label = str(load_increase) + 'x | SOH~60')
            for ii, SOC_opt in enumerate(x):
                plt.annotate(str(round((1000/60) * unserved_energy[load_increase]\
                                       [np.array(list(service_life[load_increase].keys()))[ii]][soh],
                                       round_dec_pl))+' MWh', 
                             (x[ii] + 0.01, y[ii] - y_offset), 
                             fontsize = 15)
        if soh == 'SOH_40':
            plt.plot(x, y, '-^', markersize=10, label = str(load_increase) + 'x | SOH~40')
            for ii, SOC_opt in enumerate(x):
                plt.annotate(str(round((1000/60) * unserved_energy[load_increase]\
                                       [np.array(list(service_life[load_increase].keys()))[ii]][soh],
                                       round_dec_pl))+' MWh', 
                             (x[ii] + 0.01, y[ii] - y_offset), 
                             fontsize = 15)
        
        plt.xlabel('BESS Capacity [MWh]', fontsize = 20)
        plt.ylabel('Service Years [-]', fontsize = 20)
        plt.title('Service Years [-] vs BESS Capacity [MWh] For Load Increases with Undelivered Energy Shown [MWh]', 
                  fontsize = 20)
        plt.legend(fontsize = 10, loc = 7);

In [8]:
load_increase = 1.02

BESS_capacities = list(service_life[load_increase].keys())#[:3]

x_offset = 0.01
y_offset = 1

for SOC_opt in BESS_capacities:
        
    x = list(unserved_energy_per_year[load_increase][SOC_opt].keys())
    y = (1000/60) * np.array(list(unserved_energy_per_year[load_increase][SOC_opt].values()))
    
    for ii, soh in enumerate(['SOH_80', 'SOH_60', 'SOH_40']):
        plt.annotate(soh + ' at ' + str(SOC_opt * 1000/60) + ' MWh', 
                     (service_life[load_increase][SOC_opt][soh] + x_offset, 
                      unserved_energy[load_increase][SOC_opt][soh] * 1000/60 + y_offset), 
                     fontsize = 12, 
                     rotation = 15)
        plt.plot(service_life[load_increase][SOC_opt][soh],
                 unserved_energy[load_increase][SOC_opt][soh] * 1000/60,
                 'ro') 
        
    #plt.plot(x, y, label = str(round((1000/60)*SOC_opt, 2)) + ' MWh BESS')
    plt.xlim([11.75,13.25])
    plt.ylim([0,600])
    plt.xlabel('Year [-]', fontsize = 20)
    plt.ylabel('Unserved Energy [MWh]', fontsize = 20)
    plt.title('Unserved Energy Per Year [MWh] vs Year [-] For BESS Capacities [MWh] at 80% SOH', fontsize = 20)
    #plt.legend(fontsize = 10);

In [9]:
load_increase = 1.04

BESS_capacities = list(service_life[load_increase].keys())#[:3]

x_offset = -0.15
y_offset = 2000

for SOC_opt in BESS_capacities:
        
    x = list(unserved_energy_per_year[load_increase][SOC_opt].keys())
    y = (1000/60) * np.array(list(unserved_energy_per_year[load_increase][SOC_opt].values()))
    
    #for ii, soh in enumerate(['SOH_80', 'SOH_60', 'SOH_40']):
    for ii, soh in enumerate(['SOH_60']):
#         plt.annotate(soh + ' at ' + str(SOC_opt * 1000/60) + ' MWh', (service_life[load_increase][SOC_opt][soh], 
#                            unserved_energy[load_increase][SOC_opt][soh] * 1000/60), fontsize = 12, rotation = 65)
        plt.annotate(str(SOC_opt * 1000/60) + ' MWh', 
                     (service_life[load_increase][SOC_opt][soh] + x_offset, 
                      unserved_energy[load_increase][SOC_opt][soh] * 1000/60 + y_offset), 
                     fontsize = 12, 
                     rotation = 0)
        plt.plot(service_life[load_increase][SOC_opt][soh],
                 unserved_energy[load_increase][SOC_opt][soh] * 1000/60,
                 'ro') 
        
    #plt.plot(x, y, label = str(round((1000/60)*SOC_opt, 2)) + ' MWh BESS')
    plt.xlim([14,20])
    #plt.ylim([:])
    plt.xlabel('Year [-]', fontsize = 20)
    plt.ylabel('Unserved Energy [MWh]', fontsize = 20)
    plt.title('Unserved Energy Per Year [MWh] vs Year [-] For BESS Capacities [MWh] at 60% SOH', fontsize = 20)
    #plt.legend(fontsize = 10);

In [10]:
load_increase = 1.04

BESS_capacities = list(service_life[load_increase].keys())#[:3]

x_offset = -0.05
y_offset = 2000

for SOC_opt in BESS_capacities:
        
    x = list(unserved_energy_per_year[load_increase][SOC_opt].keys())
    y = (1000/60) * np.array(list(unserved_energy_per_year[load_increase][SOC_opt].values()))
    
    #for ii, soh in enumerate(['SOH_80', 'SOH_60', 'SOH_40']):
    for ii, soh in enumerate(['SOH_40']):
#         plt.annotate(soh + ' at ' + str(SOC_opt * 1000/60) + ' MWh', (service_life[load_increase][SOC_opt][soh], 
#                            unserved_energy[load_increase][SOC_opt][soh] * 1000/60), fontsize = 12, rotation = 65)
        plt.annotate(str(SOC_opt * 1000/60) + ' MWh', 
                     (service_life[load_increase][SOC_opt][soh] + x_offset, 
                      unserved_energy[load_increase][SOC_opt][soh] * 1000/60 + y_offset), 
                     fontsize = 12, 
                     rotation = 0)
        plt.plot(service_life[load_increase][SOC_opt][soh],
                 unserved_energy[load_increase][SOC_opt][soh] * 1000/60,
                 'ro') 
        
    #plt.plot(x, y, label = str(round((1000/60)*SOC_opt, 2)) + ' MWh BESS')
    #plt.xlim([14,20])
    #plt.ylim([:])
    plt.xlabel('Year [-]', fontsize = 20)
    plt.ylabel('Unserved Energy [MWh]', fontsize = 20)
    plt.title('Unserved Energy Per Year [MWh] vs Year [-] For BESS Capacities [MWh] at 40% SOH', fontsize = 20)
    #plt.legend(fontsize = 10);